# Eniac Discount Strategy Analysis: Data Cleaning with Pandas

First, we'll go through a few data cleaning steps that should be performed on all new datasets where necessary.

We'll go through the process with Pandas DataFrames, which are `orders`, `orderlines`, and `products` datasets. 

> Pandas DataFrames are mutable two-dimensional structures of data with labeled axes where:
> * each row represents a different observation
> * each column represents a different variable


We first need to import the Pandas module:

In [1]:
import pandas as pd

Then, we define Pandas display format:

In [2]:
pd.set_option('display.max_colwidth', None) # to increase the width of the columns

## 0.&nbsp; Understanding the datasets

We will be working with 4 csv files: ``orders.csv``, ``orders.csv``, ``orders.csv``, and ``orders.csv``

Here’s a description of each table and its columns:

* ### ``orders.csv``:

  Every row in this file represents an order.
  * **order_id** – a unique identifier for each order
  * **created_date** – a timestamp for when the order was created
  * **total_paid** – the total amount paid by the customer for this order, in euros
  * **state:** 
    * “Shopping basket” – products have been placed in the shopping basket
    * “Place Order” – the order has been placed, but is awaiting shipment details 
    * “Pending” – the order is awaiting payment confirmation
    * “Completed” – the order has been placed and paid, and the transaction is completed.
    * “Cancelled” – the order has been cancelled and the payment returned to the customer.

* ### ``orderlines.csv``:

    Every row represents each one of the different products involved in an order.
    * **id** – a unique identifier for each row in this file
    * **id_order** – corresponds to orders.order_id
    * **product_id** – an old identifier for each product, nowadays not in use
    * **product_quantity** – how many units of that product were purchased on that order
    * **sku** – stock keeping unit: a unique identifier for each product
    * **unit_price** – the unitary price (in euros) of each product at the moment of placing that order
    * **date** – timestamp for the processing of that product

* ### ``products.csv``:

    * **sku** – stock keeping unit: a unique identifier for each product
    * **name** – product name
    * **desc** – product description
    * **price** – base price of the product, in euros
    * **promo_price** – promotional price, in euros
    * **in_stock** – whether or not the product was in stock at the moment of the data extraction
    * **type** – a numerical code for product type


* ### ``brands.csv``:

    * **short** – the 3-character code by which the brand can be identified in the first 3 characters of products.sku
    * **long** – brand name

## 1.&nbsp; `orders` DataFrame

### 1.1.&nbsp; Import a csv file to DataFrame

Most of the time, you will not be creating DataFrames yourself, but importing (or "reading") data from a csv file or a database into a pandas DataFrame. It's easy to do with pandas' read functions. We will read one of the datasets from the Eniac project, which we have stored in a Google Drive folder:

In [2]:
# orders.csv
url = "https://drive.google.com/file/d/16AWnkdskNpAEX5szFfNptt5xJHNWz6jk/view?usp=sharing"
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
orders = pd.read_csv(path)

Before we begin, let's create a copy of the DataFrames. This way we are sure any of our changes won't affect the original DataFrames

In [4]:
orders_df = orders.copy()

### 1.2.&nbsp; DataFrames exploration

Looking at the raw data is a great way to get a grasp of what's in there.

We have two ways to explore DataFrames:

* **Attributes:** `.shape`, `.size` and `.ndim`, and others. They are written without parentheses and give you raw "metadata" about the DataFrame you are calling them on.
* **Methods:** `.head()`, `.describe()` and `.isna()`, and others. They are written with parentheses and perform some sort of calculation, transformation or aggregation. A method is like a function that is tied to a specific object type.

In [5]:
orders_df.shape

(226909, 4)

One of the best ways to begin data cleaning is by exploring using `.info()`. This will tell us:
* The shape of the DataFrame
* The names of the columns
* If there are any missing values
* The datatypes of the columns

By exploring the missing values and correcting any incorrect datatypes, we often come across inconsistencies in our data.

Beyond this, we should also have a **check for any duplicate rows**.

In [6]:
orders_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 226909 entries, 0 to 226908
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   order_id      226909 non-null  int64  
 1   created_date  226909 non-null  object 
 2   total_paid    226904 non-null  float64
 3   state         226909 non-null  object 
dtypes: float64(1), int64(1), object(2)
memory usage: 6.9+ MB


* `total_paid` has 5 missing values
* `created_date` should become datetime datatype

In [5]:
orders_df.sample(5)

,order_id,created_date,total_paid,state
189046,489525,2018-01-12 13:53:47,3309.58,Shopping Basket
40919,340407,2017-03-30 10:54:56,79.98,Pending
212673,513164,2018-02-17 03:36:44,39.99,Completed
44377,343869,2017-04-06 10:32:09,12.99,Shopping Basket
28264,327744,2017-02-22 19:16:06,52.99,Shopping Basket


### 1.3.&nbsp; Missing values:
* `total_paid` has 5 missing values

In [9]:
orders_df["total_paid"].isna().sum()

np.int64(5)

In [10]:
print(f"5 missing values represents {((orders_df.total_paid.isna().sum() / orders_df.shape[0])*100).round(5)}% of the rows in our DataFrame")

5 missing values represents 0.0022% of the rows in our DataFrame


A quick way to find out a percentage here, if you don't need to print out a sentence is `.value_count(normalize=True)`

In [8]:
orders_df.total_paid.isna().value_counts(normalize=True)

total_paid
False    0.999978
True     0.000022
Name: proportion, dtype: float64

In [11]:
orders_df.loc[orders_df['total_paid'].isna(), :]

,order_id,created_date,total_paid,state
127701,427314,2017-11-20 18:54:39,NaN,Pending
132013,431655,2017-11-22 12:15:24,NaN,Pending
147316,447411,2017-11-27 10:32:37,NaN,Pending
148833,448966,2017-11-27 18:54:15,NaN,Pending
149434,449596,2017-11-27 21:52:08,NaN,Pending


As there is such a tiny amount of missing values, we will simply delete these rows, as we have enough data without them.

In [12]:
orders_df = orders_df.loc[~orders.total_paid.isna(), :]

In [13]:
orders_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 226904 entries, 0 to 226908
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   order_id      226904 non-null  int64  
 1   created_date  226904 non-null  object 
 2   total_paid    226904 non-null  float64
 3   state         226904 non-null  object 
dtypes: float64(1), int64(1), object(2)
memory usage: 8.7+ MB


Should you have a significant number of missing values in the future, you have a choice:
+ you can impute the values
+ you can take the values from other DataFrames, if they are present there
+ you can delete the values
+ or any number of other creative solutions

Please, always consider how much time you have on your project, and what impact your method of choice will have on your final assesment.

### 1.4.&nbsp; Duplicates

We can check for duplicates using the pandas [.duplicated()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.duplicated.html) method.

We can then delete these rows, if we wish, using [.drop_duplicates()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop_duplicates.html)

In [14]:
orders_df.duplicated().sum()

np.int64(0)

In [15]:
orders_df.duplicated().value_counts(normalize=True)

False    1.0
Name: proportion, dtype: float64

We have no duplicate rows in `orders_df` DataFrame. Easy, there is no problem to solve. Normally though, if there were some duplicates, we'd drop the extra rows.

### 1.5.&nbsp; Data types

* `created_date` should become datetime datatype

In [16]:
orders_df["created_date"] = pd.to_datetime(orders_df["created_date"])

In [17]:
orders_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 226904 entries, 0 to 226908
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   order_id      226904 non-null  int64         
 1   created_date  226904 non-null  datetime64[ns]
 2   total_paid    226904 non-null  float64       
 3   state         226904 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 8.7+ MB


### 1.6.&nbsp; Save the cleaned DataFrame

Don't forget to download/save the new DataFrames. Also, we give them an obvious name, so that we know they are the cleaned version and not the original DataFrame.

In [37]:
#files.download("orders_cl.csv")
orders_df.to_csv("orders_cl.csv", index=False)

## 2.&nbsp; `orderlines` DataFrame

### 2.1.&nbsp; Import a csv file to DataFrame

In [7]:
# orderlines.csv
url = "https://drive.google.com/file/d/1eZWrg_k0zGwW0r4RZvJw24GhchXF0tcR/view?usp=sharing"
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
orderlines = pd.read_csv(path)

First, we create a copy of the DataFrame

In [8]:
orderlines_df = orderlines.copy()

### 2.2.&nbsp; DataFrames exploration

`.info()` tells us about how pandas sees the data - how it is stored, whether there's any missing values, and how many columns and rows we have.

In [9]:
orderlines_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 293983 entries, 0 to 293982
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   id                293983 non-null  int64 
 1   id_order          293983 non-null  int64 
 2   product_id        293983 non-null  int64 
 3   product_quantity  293983 non-null  int64 
 4   sku               293983 non-null  object
 5   unit_price        293983 non-null  object
 6   date              293983 non-null  object
dtypes: int64(4), object(3)
memory usage: 15.7+ MB


* `date` should be a datetime datatype
* `unit_price` should be a float datatype

In [21]:
orderlines_df.sample(5)

,id,id_order,product_id,product_quantity,sku,unit_price,date
139709,1393704,420138,0,1,PAC2194,255.59,2017-10-31 00:01:36
276578,1622775,515520,0,1,APP2484,899.00,2018-02-21 15:53:18
291135,1645441,525166,0,1,APP0663,199.00,2018-03-10 21:50:12
88866,1289519,375854,0,1,LAC0172,189.99,2017-07-11 13:10:38
105554,1319931,390386,0,1,IFX0012,29.99,2017-08-18 10:12:24


### 2.3.&nbsp; Missing values:

There are no missing values in `orderlines_df`

### 2.4.&nbsp; Duplicates

In [22]:
orderlines_df.duplicated().sum()

np.int64(0)

In [23]:
orderlines_df.duplicated().value_counts(normalize=True)

False    1.0
Name: proportion, dtype: float64

We have no duplicate rows in `orderlines_df` DataFrame.

### 2.5.&nbsp; Data types

* `date` should be a datetime datatype
* `unit_price` should be a float datatype

#### 2.5.1.&nbsp; `date`

In [24]:
orderlines_df["date"] = pd.to_datetime(orderlines_df["date"])

In [25]:
orderlines_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 293983 entries, 0 to 293982
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   id                293983 non-null  int64         
 1   id_order          293983 non-null  int64         
 2   product_id        293983 non-null  int64         
 3   product_quantity  293983 non-null  int64         
 4   sku               293983 non-null  object        
 5   unit_price        293983 non-null  object        
 6   date              293983 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(4), object(2)
memory usage: 15.7+ MB


#### 2.5.2.&nbsp; `unit_price`

In [12]:
#orderlines_df["unit_price"] = pd.to_numeric(orderlines_df["unit_price"])
# This code doesn't work

As you can see when we try to convert `unit_price` to a numerical datatype, we receive a `ValueError` telling us that pandas doesn't understand the number `1.137.99`. This is probably because numbers cannot have 2 decimal points. Let's see if there are any other numbers like this.

In [26]:
orderlines_df["unit_price"].str.contains(r"\d+\.\d+\.\d+").value_counts()

unit_price
False    257814
True      36169
Name: count, dtype: int64

Looks like over 36000 rows in `orderlines` are affected by this problem. Let's work out how much that is as a percentage of our total data.

In [27]:
orderlines_df.unit_price.str.contains(r"\d+\.\d+\.\d+").value_counts(normalize=True)

unit_price
False    0.876969
True     0.123031
Name: proportion, dtype: float64

In [28]:
two_dot_percentage = ((orderlines_df["unit_price"].str.contains(r"\d+\.\d+\.\d+").value_counts().iloc[1] / orderlines_df.shape[0])*100).round(2)
print(f"The 2 dot problem represents {two_dot_percentage}% of the rows in our DataFrame")

The 2 dot problem represents 12.3% of the rows in our DataFrame


In [17]:
orderlines_df.shape[0]

293983

This is a bit of a tricky decision as 12.3% is a significant amount of our data... and we might even end up losing a larger portion of our data than this too. For the moment we will delete the rows as we only have 2 weeks for this project and I'd like some quick, accurate results to show. If we have time at the end, we can come back and investigate this problem further, maybe there's a solution.

> Each row of `orderlines` represents a product in an order. For example, if order number 175 contained 3 seperate products, then order 175 would have 3 rows in `orderlines`, one row for each of the products. If 2 of those products have 'normal' prices (14.99, 15.85) and 1 has a price with 2 decimal points (1.137.99), we need to remove the whole order and not just the affected row. If we only remove the row with 2 decimal places then any later analysis about products and prices could be misleading.
>
> We therefore need to find the order numbers associated with the rows that have 2 decimal points, and then remove all the associated rows.

In [29]:
two_dot_order_ids_list = orderlines_df.loc[orderlines_df.unit_price.str.contains(r"\d+\.\d+\.\d+"), "id_order"]
orderlines_df = orderlines_df.loc[~orderlines_df.id_order.isin(two_dot_order_ids_list)]

In [30]:
orderlines_df.shape[0]

216250

We still have 216250 rows in orderlines to work with. This should be more than enough for our evaluation.

Now that all of the 2 decimal point prices have been removed, let's try again to convert the column `unit_price` to the correct datatype.

In [31]:
orderlines_df["unit_price"] = pd.to_numeric(orderlines_df["unit_price"])

It worked perfectly!

In [32]:
orderlines_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 216250 entries, 0 to 293982
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   id                216250 non-null  int64         
 1   id_order          216250 non-null  int64         
 2   product_id        216250 non-null  int64         
 3   product_quantity  216250 non-null  int64         
 4   sku               216250 non-null  object        
 5   unit_price        216250 non-null  float64       
 6   date              216250 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(4), object(1)
memory usage: 13.2+ MB


### 2.6.&nbsp; Save the cleaned DataFrame

At the end, we download/save the new DataFrame:

In [23]:
#files.download("orderlines_cl.csv")
orderlines_df.to_csv("orderlines_cl.csv", index=False)

## 3.&nbsp; `products` DataFrame

### 3.1.&nbsp; Import a csv file to DataFrame

In [10]:
# products.csv
url = "https://drive.google.com/file/d/1V9qnZT4qXDxtXN_6hXjlkBnhSpbmsI2j/view?usp=sharing" 
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
products = pd.read_csv(path)

As always, we first create a copy of the DataFrame

In [11]:
products_df=products.copy()

### 3.2.&nbsp; DataFrames exploration

In [12]:
products_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19326 entries, 0 to 19325
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   sku          19326 non-null  object
 1   name         19326 non-null  object
 2   desc         19319 non-null  object
 3   price        19280 non-null  object
 4   promo_price  19326 non-null  object
 5   in_stock     19326 non-null  int64 
 6   type         19276 non-null  object
dtypes: int64(1), object(6)
memory usage: 1.0+ MB


We can see from `.info()` above that we have missing values in `desc` and `price` and `type`.

In [36]:
products_df.sample(5)

,sku,name,desc,price,promo_price,in_stock,type
16186,PAC2098,"Apple iMac 27 ""Core i5 3.5GHz Retina 5K | 64GB...",IMac desktop computer 27 inch Retina 5K RAM 64...,4099,30.740.045,0,"5,74E+15"
2493,APP0729-A,Like new - Apple Thunderbolt Cable 0.5m White,Thunderbolt Cable 0.5m for Mac,35,329.894,0,1298
12876,PAC2033,Synology DS216 + II | WD 4TB Network for Mac a...,Synology DS216 + II with 2GB of RAM and 6TB (2...,6.319.696,5.871.791,0,12175397
3675,APP1380,"Apple iMac 27 ""Core i7 Retina 5K 4GHz | 8GB | ...",IMac desktop computer 27 inch 8GB RAM 512GB Re...,3169,30.175.839,0,"5,74E+15"
4989,PAC1044,"Apple iMac 27 ""Core i5 3.3GHz Retina 5K | 16GB...",IMac desktop computer 27 inch 5K Retina i5 3.3...,3109,28.169.901,0,"5,74E+15"


### 3.3.&nbsp; Missing values:

We have missing values in:
* `desc`
* `price`
* `type`

Since our goal is to work with pric and discounts, we might not need  data from `desc` and `type`.

We have 2 choices here:
* We can quickly and easily remove these rows.
* Creative solution

#### 3.3.1.&nbsp; `desc`

In [37]:
products_df["desc"].isna().sum()

np.int64(7)

In [38]:
products_df.loc[products_df['desc'].isna(), :]

,sku,name,desc,price,promo_price,in_stock,type
16126,WDT0211-A,"Open - Purple 2TB WD 35 ""PC Security Mac hard ...",NaN,107,814.659,0,1298
16128,APP1622-A,Open - Apple Smart Keyboard Pro Keyboard Folio...,NaN,1.568.206,1.568.206,0,1298
17843,PAC2334,Synology DS718 + NAS Server | 10GB RAM,NaN,566.35,5.659.896,0,12175397
18152,KAN0034-A,Open - Kanex USB-C Gigabit Ethernet Adapter Ma...,NaN,29.99,237.925,0,1298
18490,HTE0025,Hyper Pearl 1600mAh battery Mini USB Mirror an...,NaN,24.99,22.99,1,1515
18612,OTT0200,OtterBox External Battery Power Pack 20000 mAHr,NaN,79.99,56.99,1,1515
18690,HOW0001-A,Open - Honeywell thermostat Lyric zonificador ...,NaN,199.99,1.441.174,0,11905404


* Creative solution:

since missing values in desc are not important to us, we replace the description with the name to fill the missing values and prevent losing other data in rows with empty desc. (We could also leave them empty as they were)

In [39]:
products_df.loc[products_df['desc'].isna(), 'desc'] = products_df.loc[products_df['desc'].isna(), 'name']

Let's look at two example:

In [40]:
products_df.loc[[16126, 17843]]

,sku,name,desc,price,promo_price,in_stock,type
16126,WDT0211-A,"Open - Purple 2TB WD 35 ""PC Security Mac hard ...","Open - Purple 2TB WD 35 ""PC Security Mac hard ...",107,814.659,0,1298
17843,PAC2334,Synology DS718 + NAS Server | 10GB RAM,Synology DS718 + NAS Server | 10GB RAM,566.35,5.659.896,0,12175397


#### 3.3.2.&nbsp; `price`

In [41]:
products_df["price"].isna().sum()

np.int64(46)

In [42]:
print(f"The missing values in price are {(products_df["price"].isna().value_counts(normalize=True).iloc[1] * 100).round(2)}% of all rows in the DataFrame")

The missing values in price are 0.24% of all rows in the DataFrame


In [43]:
products_df["price"].isna().value_counts(normalize=True)

price
False    0.99762
True     0.00238
Name: proportion, dtype: float64

Let's simply delete these rows to ensure that we can trust the numbers in our final DataFrame. Afterall, the price is very important when investigating discounts.

Delete missing values:

Option 1: `.loc`
* products_df = products_df.loc[~products['price'].isna()]

Option 2: `.dropna()`
* products_df = products_df.dropna(subset=['price'])

In [44]:
products_df = products_df.loc[~products['price'].isna()]

In [45]:
products_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19280 entries, 0 to 19325
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   sku          19280 non-null  object
 1   name         19280 non-null  object
 2   desc         19280 non-null  object
 3   price        19280 non-null  object
 4   promo_price  19280 non-null  object
 5   in_stock     19280 non-null  int64 
 6   type         19230 non-null  object
dtypes: int64(1), object(6)
memory usage: 1.2+ MB


### 3.4.&nbsp; Duplicates

In [46]:
products_df.duplicated().sum()

np.int64(8746)

In [47]:
products_df.duplicated().value_counts(normalize=True)

False    0.546369
True     0.453631
Name: proportion, dtype: float64

Wow, that's a lot of duplicates. Let's get rid of them.

In [48]:
products_df = products_df.drop_duplicates()

In [49]:
products_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10534 entries, 0 to 19325
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   sku          10534 non-null  object
 1   name         10534 non-null  object
 2   desc         10534 non-null  object
 3   price        10534 non-null  object
 4   promo_price  10534 non-null  object
 5   in_stock     10534 non-null  int64 
 6   type         10484 non-null  object
dtypes: int64(1), object(6)
memory usage: 658.4+ KB


### 3.5.&nbsp; Data types

We saw from looking at the output of `.info()` that both `price` and `promo_price` have been stored as objects and not as a numerical datatypes. We also saw while solving other problems that both columns have some prices with 3 decimal places and others with 2 decimal points - the latter will prevent us from converting the datatype to numerical, so first we must investigate and solve these problems.

In [50]:
products_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10534 entries, 0 to 19325
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   sku          10534 non-null  object
 1   name         10534 non-null  object
 2   desc         10534 non-null  object
 3   price        10534 non-null  object
 4   promo_price  10534 non-null  object
 5   in_stock     10534 non-null  int64 
 6   type         10484 non-null  object
dtypes: int64(1), object(6)
memory usage: 658.4+ KB


#### 3.5.1.&nbsp; `price`

First, let's see how many values are affected by the 2-decimal-dot problems or 3 decimal places.

In [51]:
products['price'].str.count(r'\.').value_counts(normalize=True)

price
0.0    0.597925
1.0    0.379720
2.0    0.022355
Name: proportion, dtype: float64

In [52]:
(products_df
.loc
[(products_df["price"].astype(str).str.contains(r"\d+\.\d+\.\d+")) 
 |(products_df["price"].astype(str).str.contains(r"\d+\.\d{3,}")), :]).sample(3)

,sku,name,desc,price,promo_price,in_stock,type
15979,PAC1970,Synology DS1817 + | 8GB RAM | 48TB (8x6TB) Sea...,NAS with 8GB RAM and 48TB for Mac and PC,33.317.885,25.747.457,0,12175397
11410,DLK0132,D-Link DCH-100KT Smart Home Monitoring Kit HD ...,HD surveillance camera kit with intelligent mo...,1.699.905,1.699.905,0,9094
1393,APP0879,Apple iPad Air 2 Wi-Fi + Cellular 128GB Silver,New iPad Air 2 Wi-Fi + Cellular 128GB (MGWM2TY...,6.628.114,6.628.114,0,42931714


In [53]:
price_problems_number = (products_df
                         .loc[(products_df["price"].astype(str).str.contains(r"\d+\.\d+\.\d+"))
                              |(products_df["price"].astype(str).str.contains(r"\d+\.\d{3,}")), :]
                              .shape[0])
price_problems_number

542

In [54]:
print(f"The column price has in total {price_problems_number} wrong values. This is {round(((price_problems_number / products_df.shape[0]) * 100), 2)}% of the rows of the DataFrame")

The column price has in total 542 wrong values. This is 5.15% of the rows of the DataFrame


The price column will be important to understanding discounts and we are basing business decisions on the price. Therefore, we'll delete these rows with wrong values.

In [55]:
products_df = products_df.loc[(~products_df["price"].astype(str).str.contains(r"\d+\.\d+\.\d+"))&(~products_df["price"].astype(str).str.contains(r"\d+\.\d{3,}")), :]


Then, we can convert the column to a numeric datatype.

In [56]:
products_df["price"] = pd.to_numeric(products_df["price"])

In [57]:
products_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9992 entries, 0 to 19325
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   sku          9992 non-null   object 
 1   name         9992 non-null   object 
 2   desc         9992 non-null   object 
 3   price        9992 non-null   float64
 4   promo_price  9992 non-null   object 
 5   in_stock     9992 non-null   int64  
 6   type         9946 non-null   object 
dtypes: float64(1), int64(1), object(5)
memory usage: 624.5+ KB


#### 3.5.2.&nbsp; `promo_price`

Again, let's begin by seeing how many values are affected by the 2-decimal-dots problem, or the 3 decimal-places problem

In [58]:
promo_problems_number = products_df.loc[(products_df["promo_price"].str.contains(r"\d+\.\d+\.\d+"))|(products_df["promo_price"].str.contains(r"\d+\.\d{3,}")), :].shape[0]
promo_problems_number

9232

In [59]:
print(f"The column promo_price has in total {promo_problems_number} wrong values. This is {round(((promo_problems_number / products_df.shape[0]) * 100), 2)}% of the rows of the DataFrame")

The column promo_price has in total 9232 wrong values. This is 92.39% of the rows of the DataFrame


WOW!!! That's a lot of wrong data. Let's have a quick investigate to check that's correct. We'll make a DataFrame by copy-pasting the code we used above and then look at a large sample to check that all the numbers in the `promo_price` column really have either 2 decimal points or 3 decimal places.

In [60]:
promo_price_df = products_df.loc[(products_df["promo_price"].astype(str).str.contains(r"\d+\.\d+\.\d+"))|(products_df["promo_price"].astype(str).str.contains(r"\d+\.\d{3,}")), :]
promo_price_df.sample(5)

,sku,name,desc,price,promo_price,in_stock,type
16135,APP1450,Apple iPad Case Silicone Case Mini 4 Rosa,Protective cover for cases ultrafine 4 mini si...,69.00,640.005,0,12635403
18495,ICA0098,"Incase backpack Reform MacBook 15 """,Incase backpack with special compartment Tensa...,149.95,1.399.897,1,1392
18546,AP20434,"Like New - Apple Macbook Pro 13 ""Core i5 23GHz...",MacBook reconditioned 13 inch Dual-core i5 23G...,1505.59,12.255.945,0,1282
10942,FCM0039-2,Mac memory FCM 32GB (2x16GB) DDR3 1867MHz SO-DIMM,Mac memory FCM 32GB (2x16GB) DDR3 1867MHz SO-D...,489.98,3.629.903,0,1364
2377,SPE0157,Speck CandyShell Grip Case for iPhone 6 White ...,ultra-resistant rubber with non-slip lines iPh...,24.99,219.857,0,11865403


Since over 90% of the data in this column is corrupted, there's no point deleting all of these rows, then we would barely have a products table. Instead, as it's only this column that appears to be very untrustworthy, we will delete the column.

> For calculating discount, we will need 2 source of price: "price" and "unit_price"

In [61]:
products_cl = products_df.drop(columns=["promo_price"])

In [62]:
products_cl.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9992 entries, 0 to 19325
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   sku       9992 non-null   object 
 1   name      9992 non-null   object 
 2   desc      9992 non-null   object 
 3   price     9992 non-null   float64
 4   in_stock  9992 non-null   int64  
 5   type      9946 non-null   object 
dtypes: float64(1), int64(1), object(4)
memory usage: 546.4+ KB


### 3.6.&nbsp; Save the cleaned DataFrame

At the end, we download/save the new DataFrame:

In [33]:
#files.download("products_cl.csv")
products_cl.to_csv("products_cl.csv", index=False)